## Data Input Notebook

In which we:
-   Take input data from a .csv file
-   Create SQL database
-   Convert input data to N-body units
-   Add data to database


In [1]:
# Import modules
import pandas as pd
import numpy as np

In [2]:
# Let's define some constants
class Constants:
    def __init__(self):
        self.m_sun = 1.988435E30 # Solar mass
        self.r_pc = 3.086E16 # 1 pc in m
        self.r_au = 1.496E11 # 1 AU in m
        self.G = 6.674E-11 # Gravitational constant
        self.v_kms = 1000. # km/s in m/s
        self.twopi = np.pi*2 # double pi
        self.t_yr = 3.154E7 # 1 yr in s
        self.t_kyr = 3.154E10 # 1 kyr in s
        self.t_myr = 3.154E13 # 1 Myr in s

In [3]:
def calculateTotalEnergy(m, r, v, G=1.):
    """
    Calculate Total Energy
    
    This function calculates the total energy (kinetic energy
    plus gravitational potential energy) of a group of objects.

    Arguments:
        `m` -- list of masses
        `r` -- list of positions
        `v' -- list of velocities
        `G' -- The gravitational constant (6.674E-11, or set to unity in N-body units)
    """
    # kinetic energy:
    e_kinetic = 0.5*np.sum( m*np.sum(v*v, axis=1) )
    # potential energy:
    e_potential = 0.
    for i in range(N):
        e_potential -= np.sum([G*m[i]*m[j]/np.sqrt(np.dot(r[i]-r[j], r[i]-r[j])) for j in range(i+1, N)])
    e_total = e_kinetic + e_potential
    return e_kinetic, e_potential, e_total

In [5]:
def calculateNbodyUnits(m, r, v, G):
    """
    Calculate N-body Units
    
    This function calculates the N-body modifiers needed to convert
    the positions, velocities and masses of a group of objects into 
    N-body units

    Arguments:
        `m` -- list of masses
        `r` -- list of positions
        `v' -- list of velocities
        `G' -- The gravitational constant (6.674E-11)
    """
    e_kinetic, e_potential, e_total = calculateTotalEnergy(m, r, v, G)
    u_m = np.sum(m)
    u_l = G*M**2 / (-4*e_total)
    u_t = G*(np.power(M,2.5)) / np.power((-4*e_total),1.5)
    u_v = u_l / u_t
    return u_m, u_l, u_t, u_v

Take the input data from a .csv file.
I’m using the Sun and the 8 planets in the solar system.
Each row contains:
-   The name
-   The mass in kg
-   The x, y and z positions in m
-   The x, y and z velocities in m/s

In [14]:
planets = pd.read_csv("../data/planets_example.csv")

In [15]:
planets

,Name,Mass,x,y,z,vx,vy,vz
0,Sun,1.990000e+30,3.330000e+08,8.560000e+08,-19500000,-9.02,9.53,0.211
1,Mercury,3.300000e+23,-5.110000e+10,-4.070000e+10,1300000000,20600.00,-35700.00,-4810.000
2,Venus,4.870000e+24,-9.450000e+10,5.110000e+10,6140000000,-16600.00,-31100.00,529.000
3,Earth,5.970000e+24,1.380000e+11,5.810000e+10,-22600000,-11900.00,27400.00,-1.970
4,Mars,6.420000e+23,-2.330000e+11,8.740000e+10,7530000000,-7520.00,-20600.00,-248.000
5,Jupiter,1.900000e+27,-6.860000e+11,-4.360000e+11,17200000000,6850.00,-10400.00,-110.000
6,Saturn,5.680000e+26,-5.360000e+10,-1.500000e+12,28300000000,9120.00,-376.00,-356.000
7,Uranus,8.680000e+25,2.670000e+12,1.320000e+12,-29700000000,-3060.00,5790.00,61.300
8,Neptune,1.020000e+26,4.280000e+12,-1.320000e+12,-71500000000,1570.00,5230.00,-144.000
